In [1]:
v = [
    (8, (1,2,3)),
    (9, (4,5)),
    (3, (6,7)),
    (6, (8,9,10)),
    (8, (11,)),
    (8, tuple()),
    (4, (12,)),
    (4, (13,)),
    (4, tuple()),
    (6, (14,15)),
    (2, tuple()),
    (4, tuple()),
    (6, (16,17,18,19)),
    (4, (20,)),
    (1, (21,)),
    (2, tuple()),
    (5, tuple()),
    (4, tuple()),
    (3, (22,)),
    (10, tuple()),
    (4, tuple()),
    (4, tuple()),
    (9, tuple())
]

In [81]:
tree
'''
8
|-9
| |-8
| +-8
|   +-4
|-3
| |-4
| | +-6
| |   |-5
| |   |-4
| |   |-3
| |   | +-9
| |   +-10
| +-4
|   +-4
|     +-4
+-6
| |-4
| |-6
| | |-1
| | | +-4
| | +-2
| +-2
'''

'\n8\n|-9\n| |-8\n| +-8\n|   +-4\n|-3\n| |-4\n| | +-6\n| |   |-5\n| |   |-4\n| |   |-3\n| |   | +-9\n| |   +-10\n| +-4\n|   +-4\n|     +-4\n+-6\n| |-4\n| |-6\n| | |-1\n| | | +-4\n| | +-2\n| +-2\n'

In [2]:
def vertex_cover_in_path_graphs(path):
    assert len(path) >= 2
    memo = [(path[1], path[0])]
    for i in range(2, len(path)):
        memo.append((memo[-1][1] + path[i], min(memo[-1][1] + path[i-1], memo[-1][0])))
    print(memo)
    result, i, m = [], len(memo)-1, min(memo[-1])
    while i >= 0:
        if memo[i][0] == m:
            result.append(i+1)
            m -= path[i+1]
            i -= 1
        else:
            result.append(i)
            m -= path[i]
            i -= 2
    result.reverse()
    return tuple(result)

In [3]:
'''
1 -> 2 -> 3 -> 4 -> 1 -> 1 -> 4
... V[n-2] -> V[n-1] -> V[n]

Observation:
Either V[n] or V[n-1] is in the vertex cover

Proof:
By contradiction: if neither V[n] nor V[n-1] is in the cover, Edge(n-1, n) is not covered. 

When we append a new Edge[n, n+1] to the graph, we have two possibilities: either V[n] or V[n+1]
is part of the cover. 

Adding V[n] to the graph gives us three options:

1. Include V[n] to the cover that does not include V[n-1] (101)
2. Exclude V[n], and choose the minimum of the two possible covers that include V[n-1] (110, 010)

010
110
101
'''

vertex_cover_in_path_graphs((4,1,1,4,1))

[(1, 4), (5, 1), (5, 2), (3, 5)]


(1, 2, 4)

In [45]:
def vertex_cover_in_trees(tree):
    '''
    Depth-first search (using a stack rather than recursion)
    '''
    visited, vertexstack, intersections = {0}, [0], dict()

    def set_cover_score():
        '''
        Method that sets the cover scores for the path starting
        at the 'lowest' vertex in the stack, upto the first inter-
        section.
        '''
        start, path = vertexstack[-1], list()
        # Find the path upto the first intersection
        while True:
            path.append(tree[vertexstack.pop()][0])
            if len(tree[vertexstack[-1]][1]) > 1:
                path.append(tree[vertexstack[-1]][0])
                break

        # Initialize the memo to the last known covers, if any
        if start in intersections:
            memo, i = [intersections[start]], 1
        else:
            memo, i = [(path[1], path[0])], 2

        # Calculate the new covers
        for i in range(i, len(path)):
            memo.append((memo[-1][1] + path[i], min(memo[-1][1] + path[i-1], memo[-1][0])))

        # Set the cover at the intersection
        if vertexstack[-1] in intersections:
            # We have already calculated the cover score of this intersection
            # for another of its child paths. We just need to update the cover
            # score, keeping in mind that we don't count the intersection's
            # weight double.
            intersections[vertexstack[-1]] = (
                intersections[vertexstack[-1]][0] + memo[-1][0] - path[-1],
                intersections[vertexstack[-1]][1] + memo[-1][1]
            )
        else:
            # Set the cover score of this intersection based on the child path
            intersections[vertexstack[-1]] = memo[-1]

    # Here we do our depth first search through the tree:
    while len(vertexstack) > 0:
        cur_vertex = vertexstack[-1]
        # Add the first child that has not been visited yet
        for c in tree[vertexstack[-1]][1]:
            if c not in visited:
                vertexstack.append(c)
                visited.add(c)
                break
        # if there are no more children that have not been visited, then
        # calculate the cover score of the bottom-most vertex upto the first
        # intersection
        if cur_vertex == vertexstack[-1]:
            if len(vertexstack) > 1:
                set_cover_score()
            else: 
                # UNLESS: there is only a single node left, in which case
                # we pop it and end the search
                vertexstack.pop()

    # Retun the minimum score of the root's cover
    return min(intersections[0])

In [46]:
vertex_cover_in_trees(v)

44